In [ ]:
!unzip -q '/content/drive/MyDrive/Colab Notebooks/MIPT/gan-telegrambot/img_align_celeba.zip'

In [ ]:
import argparse
import os
import numpy as np
import math
import itertools
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader
from torch.autograd import Variable

from models import *
from datasets import *

import torch.nn as nn
import torch.nn.functional as F
import torch

os.makedirs("images/training", exist_ok=True)
os.makedirs("saved_models", exist_ok=True)

In [ ]:
# parser = argparse.ArgumentParser()
# parser.add_argument("-epoch", type=int, default=0, help="epoch to start training from")
# parser.add_argument("-n_epochs", type=int, default=200, help="number of epochs of training")
# parser.add_argument("-dataset_name", type=str, default="img_align_celeba", help="name of the dataset")
# parser.add_argument("-batch_size", type=int, default=4, help="size of the batches")
# parser.add_argument("-lr", type=float, default=0.0002, help="adam: learning rate")
# parser.add_argument("-b1", type=float, default=0.9, help="adam: decay of first order momentum of gradient")
# parser.add_argument("-b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
# parser.add_argument("-decay_epoch", type=int, default=100, help="epoch from which to start lr decay")
# parser.add_argument("-n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
# parser.add_argument("-hr_height", type=int, default=256, help="high res. image height")
# parser.add_argument("-hr_width", type=int, default=256, help="high res. image width")
# parser.add_argument("-channels", type=int, default=3, help="number of image channels")
# parser.add_argument("-sample_interval", type=int, default=100, help="interval between saving image samples")
# parser.add_argument("-checkpoint_interval", type=int, default=5000, help="batch interval between model checkpoints")
# parser.add_argument("-residual_blocks", type=int, default=23, help="number of residual blocks in the generator")
# parser.add_argument("-warmup_batches", type=int, default=500, help="number of batches with pixel-wise loss only")
# parser.add_argument("-lambda_adv", type=float, default=5e-3, help="adversarial loss weight")
# parser.add_argument("-lambda_pixel", type=float, default=1e-2, help="pixel-wise loss weight")
# opt = parser.parse_args()

In [ ]:
class Opt():
  def __init__(self):
    self.epoch = 0
    self.n_epochs = 200
    self.dataset_name = "img_align_celeba"
    self.batch_size = 4
    self.lr = 0.0002
    self.b1 = 0.9
    self.b2 = 0.999
    self.decay_epoch = 100
    self.n_cpu = 8
    self.hr_height = 256
    self.hr_width = 256
    self.channels = 3
    self.sample_interval = 100
    self.checkpoint_interval = 5000
    self.residual_blocks = 23
    self.warmup_batches = 500
    self.lambda_adv = 5e-3
    self.lambda_pixel = 1e-2
opt = Opt()
print(opt.__dict__)

{'epoch': 0, 'n_epochs': 200, 'dataset_name': 'img_align_celeba', 'batch_size': 4, 'lr': 0.0002, 'b1': 0.9, 'b2': 0.999, 'decay_epoch': 100, 'n_cpu': 8, 'hr_height': 256, 'hr_width': 256, 'channels': 3, 'sample_interval': 100, 'checkpoint_interval': 5000, 'residual_blocks': 23, 'warmup_batches': 500, 'lambda_adv': 0.005, 'lambda_pixel': 0.01}


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

hr_shape = (opt.hr_height, opt.hr_width)

In [ ]:
# Initialize generator and discriminator
generator = GeneratorRRDB(opt.channels, filters=64, num_res_blocks=opt.residual_blocks).to(device)
discriminator = Discriminator(input_shape=(opt.channels, *hr_shape)).to(device)
feature_extractor = FeatureExtractor().to(device)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
# Set feature extractor to inference mode
feature_extractor.eval()

FeatureExtractor(
  (vgg19_54): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride

In [ ]:
# Losses
criterion_GAN = torch.nn.BCEWithLogitsLoss().to(device)
criterion_content = torch.nn.L1Loss().to(device)
criterion_pixel = torch.nn.L1Loss().to(device)

In [ ]:
if opt.epoch != 0:
    # Load pretrained models
    generator.load_state_dict(torch.load("saved_models/generator_%d.pth" % opt.epoch))
    discriminator.load_state_dict(torch.load("saved_models/discriminator_%d.pth" % opt.epoch))

In [ ]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

In [ ]:
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor

In [ ]:
dataloader = DataLoader(
    ImageDataset("/content/%s" % opt.dataset_name, hr_shape=hr_shape),
    batch_size=opt.batch_size,
    shuffle=True,
    num_workers=opt.n_cpu,
)

/usr/local/lib/python3.8/dist-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# ----------
#  Training
# ----------

for epoch in range(opt.epoch, opt.n_epochs):
    for i, imgs in enumerate(dataloader):

        batches_done = epoch * len(dataloader) + i

        # Configure model input
        imgs_lr = Variable(imgs["lr"].type(Tensor))
        imgs_hr = Variable(imgs["hr"].type(Tensor))

        # Adversarial ground truths
        valid = Variable(Tensor(np.ones((imgs_lr.size(0), *discriminator.output_shape))), requires_grad=False)
        fake = Variable(Tensor(np.zeros((imgs_lr.size(0), *discriminator.output_shape))), requires_grad=False)

        # ------------------
        #  Train Generators
        # ------------------

        optimizer_G.zero_grad()

        # Generate a high resolution image from low resolution input
        gen_hr = generator(imgs_lr)

        # Measure pixel-wise loss against ground truth
        loss_pixel = criterion_pixel(gen_hr, imgs_hr)

        if batches_done < opt.warmup_batches:
            # Warm-up (pixel-wise loss only)
            loss_pixel.backward()
            optimizer_G.step()
            print(
                "[Epoch %d/%d] [Batch %d/%d] [G pixel: %f]"
                % (epoch, opt.n_epochs, i, len(dataloader), loss_pixel.item())
            )
            continue

        # Extract validity predictions from discriminator
        pred_real = discriminator(imgs_hr).detach()
        pred_fake = discriminator(gen_hr)

        # Adversarial loss (relativistic average GAN)
        loss_GAN = criterion_GAN(pred_fake - pred_real.mean(0, keepdim=True), valid)

        # Content loss
        gen_features = feature_extractor(gen_hr)
        real_features = feature_extractor(imgs_hr).detach()
        loss_content = criterion_content(gen_features, real_features)

        # Total generator loss
        loss_G = loss_content + opt.lambda_adv * loss_GAN + opt.lambda_pixel * loss_pixel

        loss_G.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        pred_real = discriminator(imgs_hr)
        pred_fake = discriminator(gen_hr.detach())

        # Adversarial loss for real and fake images (relativistic average GAN)
        loss_real = criterion_GAN(pred_real - pred_fake.mean(0, keepdim=True), valid)
        loss_fake = criterion_GAN(pred_fake - pred_real.mean(0, keepdim=True), fake)

        # Total loss
        loss_D = (loss_real + loss_fake) / 2

        loss_D.backward()
        optimizer_D.step()

        # --------------
        #  Log Progress
        # --------------

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f, content: %f, adv: %f, pixel: %f]"
            % (
                epoch,
                opt.n_epochs,
                i,
                len(dataloader),
                loss_D.item(),
                loss_G.item(),
                loss_content.item(),
                loss_GAN.item(),
                loss_pixel.item(),
            )
        )

        if batches_done % opt.sample_interval == 0:
            # Save image grid with upsampled inputs and ESRGAN outputs
            imgs_lr = nn.functional.interpolate(imgs_lr, scale_factor=4)
            img_grid = denormalize(torch.cat((imgs_lr, gen_hr), -1))
            save_image(img_grid, "images/training/%d.png" % batches_done, nrow=1, normalize=False)

        if batches_done % opt.checkpoint_interval == 0:
            # Save model checkpoints
            torch.save(generator.state_dict(), "saved_models/generator_%d.pth" % epoch)
            torch.save(discriminator.state_dict(), "saved_models/discriminator_%d.pth" %epoch)

Streaming output truncated to the last 5000 lines.
[Epoch 0/200] [Batch 4461/50650] [D loss: 0.000763] [G loss: 0.966056, content: 0.919864, adv: 8.940087, pixel: 0.149090]
[Epoch 0/200] [Batch 4462/50650] [D loss: 0.000113] [G loss: 0.954845, content: 0.897728, adv: 11.112742, pixel: 0.155315]
[Epoch 0/200] [Batch 4463/50650] [D loss: 0.000853] [G loss: 1.030262, content: 0.985294, adv: 8.701346, pixel: 0.146132]
[Epoch 0/200] [Batch 4464/50650] [D loss: 0.003015] [G loss: 0.946918, content: 0.908429, adv: 7.394756, pixel: 0.151480]
[Epoch 0/200] [Batch 4465/50650] [D loss: 0.001139] [G loss: 1.202277, content: 1.156013, adv: 8.925406, pixel: 0.163770]
[Epoch 0/200] [Batch 4466/50650] [D loss: 0.000432] [G loss: 1.024077, content: 0.968946, adv: 10.692608, pixel: 0.166711]
[Epoch 0/200] [Batch 4467/50650] [D loss: 0.223418] [G loss: 1.321802, content: 1.303129, adv: 3.301370, pixel: 0.216596]
[Epoch 0/200] [Batch 4468/50650] [D loss: 0.007320] [G loss: 1.084378, content: 1.054206, adv